### Geolocalisation using ChatGPT

### Packages

In [1]:
import base64
import os
import re

import cv2
import geopandas as gpd
import pandas as pd
from folium import Map
from openai import AzureOpenAI
from tqdm import tqdm

### Functions

In [5]:
def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal = degrees + minutes / 60 + seconds / 3600
    if direction in ["S", "W"]:
        decimal *= -1
    return decimal


def parse_dms(dms_list):
    # Join list into one big string
    dms_string = "\n".join(dms_list)

    pattern = r"(\d+)°(\d+)'(\d+(?:\.\d+)?)\" ([NSEW])"
    matches = re.findall(pattern, dms_string)

    latitudes = []
    longitudes = []

    for i in range(0, len(matches), 2):  # Each pair: lat, lon
        lat_deg, lat_min, lat_sec, lat_dir = matches[i]
        lon_deg, lon_min, lon_sec, lon_dir = matches[i + 1]

        lat = dms_to_decimal(int(lat_deg), int(lat_min), float(lat_sec), lat_dir)
        lon = dms_to_decimal(int(lon_deg), int(lon_min), float(lon_sec), lon_dir)

        latitudes.append(lat)
        longitudes.append(lon)

    return latitudes, longitudes


def geolocate_fig(file_path):
    # Define prompt
    prompt = [
        "You are an expert geoguesser who can accurately identify locations from street view images.",
        "Analyze the provided image and determine the most likely location.",
        "Be as precise as possible. If you are unsure, make your best educated guess.",
        "Return your answer strictly in the following JSON format:",
        '{"city": "city name", "country": "country name", "latitude": latitude, "longitude": longitude}',
        "Do not include any explanations or extra text, only the JSON object without a code block.",
    ]

    prompt = " ".join(prompt)

    # Read figure
    return responde_to_fig(file_path, prompt)


def responde_to_fig(file_path, prompt):
    # Read figure
    fig = cv2.imread(file_path)

    # Encode figure
    retval, buffer = cv2.imencode(os.path.splitext(file_path)[1], fig)
    if not retval:
        raise ValueError("Failed to convert image")
    fig_encoded = base64.b64encode(buffer).decode("utf-8")

    # Load environment variables
    api_version = "2024-03-01-preview"
    api_base_url = os.getenv("OPENAI_API_BASE")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

    # Initialise large language model
    model = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        base_url=f"{api_base_url}/deployments/{deployment_name}",
    )

    # Trigger model
    response = model.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{fig_encoded}", "detail": "high"}},
                ],
            }
        ],
        max_tokens=1000,
        temperature=0,
    )

    # Return response
    return response.choices[0].message.content

### Settings

In [6]:
# File paths
dir_base = r"p:\11211481-image-registration\02_exploratory_geoguessr"
file_path_locations = os.path.join(dir_base, "01_data", "coordinates.xlsx")
dir_path_images = os.path.join(dir_base, "01_data", "01_images")

# Read locations
df_locations = pd.read_excel(file_path_locations)

# Get latitudes and longitudes from DMS coordinates
latitudes, longitudes = parse_dms(df_locations["coordinates"].tolist())
df_locations["latitude"] = latitudes
df_locations["longitude"] = longitudes

# Convert to GeoDataFrame
gdf_locations = gpd.GeoDataFrame(df_locations, geometry=gpd.points_from_xy(df_locations.longitude, df_locations.latitude), crs="EPSG:4326")

# Display locations
df_locations.head()

,ID,city,country,coordinates,latitude,longitude
0,1,Valencia,Spain,"39°27'41.31"" N 0°22'18.50"" W",39.461475,-0.371806
1,2,Garissa,Kenya,"0°26'24.21"" S 39°39'28.86"" E",-0.440058,39.658017
2,3,Chiang Mai,Thailand,"18°47'08.82"" N 99°01'14.49"" E",18.785783,99.020692
3,4,Beijing,China,"39°46'38.40"" N 116°47'19.61"" E",39.777333,116.788781
4,5,Malyy Istok,Russia,"56°46'30.96"" N 60°50'49.26"" E",56.775267,60.847017


### Geolocalisation

In [7]:
# Get results
responses = []
for _, row in tqdm(df_locations.iterrows(), total=len(df_locations)):
    # Get image file path
    file_path_image = os.path.join(dir_path_images, f"{row["ID"]:03d}.jpg")

    # Check if file exists
    if not os.path.exists(file_path_image):
        print(os.path.basename(file_path_image), "does not exist. Skipping.")
        continue

    # Geolocate figure
    response = geolocate_fig(file_path_image)

    # Convert JSON string to dictionary
    response = eval(response)

    # Append ID to response
    response = {"ID": row["ID"], **response}
    
    # Add response to list
    responses.append(response)

# Convert responses to DataFrame
df_responses = pd.DataFrame(responses)

# Convert to GeoDataFrame
gdf_responses = gpd.GeoDataFrame(df_responses, geometry=gpd.points_from_xy(df_responses.longitude, df_responses.latitude), crs="EPSG:4326")

# Display locations
gdf_responses.head()

100%|██████████| 7/7 [00:30<00:00,  4.31s/it]


,ID,city,country,latitude,longitude,geometry
0,1,Valencia,Spain,39.4699,-0.3763,POINT (-0.3763 39.4699)
1,2,Lodwar,Kenya,3.1191,35.5973,POINT (35.5973 3.1191)
2,3,Chiang Mai,Thailand,18.7883,98.9853,POINT (98.9853 18.7883)
3,4,Harbin,China,45.8038,126.5347,POINT (126.5347 45.8038)
4,5,Novosibirsk,Russia,55.0084,82.9357,POINT (82.9357 55.0084)


### Explore locations

In [8]:
# Explore locations
m = Map()
gdf_locations.explore(m=m, name="Locations", color="blue", marker_kwds={"radius": 5})
gdf_responses.explore(m=m, name="Responses", color="red", marker_kwds={"radius": 5})
m